In [1]:
%autoawait
import time
import asyncio
import base64

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from om_aries_controller.om_aries_controller import OMAriesController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

# Based on the aca-py agent you wish to control
om_controller = OMAriesController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

In [3]:
response = await om_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")


Results :  [{'invitation_mode': 'once', 'invitation_key': '27mkkudkjHrZQ56QCM9XvLsjjoTVJJgt2ymGuRYwUrYd', 'my_did': 'WuBEbhuAkrRjUPXvxMRyTc', 'routing_state': 'none', 'state': 'active', 'their_label': 'Alice', 'updated_at': '2020-09-10 05:17:08.852843Z', 'accept': 'manual', 'connection_id': '8a5213fb-f170-4cc8-896a-765d9b90457e', 'initiator': 'self', 'their_did': 'GXGsuZJxFiSExKwFfbNKgg', 'created_at': '2020-09-10 05:16:44.823775Z'}]
Connection : {'invitation_mode': 'once', 'invitation_key': '27mkkudkjHrZQ56QCM9XvLsjjoTVJJgt2ymGuRYwUrYd', 'my_did': 'WuBEbhuAkrRjUPXvxMRyTc', 'routing_state': 'none', 'state': 'active', 'their_label': 'Alice', 'updated_at': '2020-09-10 05:17:08.852843Z', 'accept': 'manual', 'connection_id': '8a5213fb-f170-4cc8-896a-765d9b90457e', 'initiator': 'self', 'their_did': 'GXGsuZJxFiSExKwFfbNKgg', 'created_at': '2020-09-10 05:16:44.823775Z'}
Active Connection ID :  8a5213fb-f170-4cc8-896a-765d9b90457e


In [4]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    with open("test.txt","w+") as f
    text = base64.b64decode(data)
    f.write(data)
#     data = payload['content'] 
#     asyncio.get_event_loop().create_task(om_controller.messaging.send_message(connection_id, " Received!"))
message_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([message_listener], defaults=True)


Handle Attachment {'connection_id': '8a5213fb-f170-4cc8-896a-765d9b90457e', 'message_id': 'cb18a36a-d941-4135-8085-efa9d4dd4082', 'attachment_message': 'Here is the data you wanted', 'content': 'VGhpcyBpcyBhIHRlc3QgZmlsZSB0byBhdHRhY2gu', 'filename': 'test_file.txt', 'state': 'received'} 8a5213fb-f170-4cc8-896a-765d9b90457e


In [10]:
response = await om_controller.protocol.send_attachment(connection_id, data)

print(response)

NameError: name 'data' is not defined

In [13]:
response = await om_controller.terminate()
print(response)


None


ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<MessagingController.send_message() done, defined at /aries-basic-controller/aries_basic_controller/messaging_controller.py:16> exception=RuntimeError('Session is closed')>
Traceback (most recent call last):
  File "/aries-basic-controller/aries_basic_controller/messaging_controller.py", line 18, in send_message
    "content": msg,
  File "/aries-basic-controller/aries_basic_controller/base_controller.py", line 96, in admin_POST
    response = await self.admin_request("POST", path, json_data, text, params, data)
  File "/aries-basic-controller/aries_basic_controller/base_controller.py", line 61, in admin_request
    method, self.admin_url + path, json=json_data, params=params, data=data
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/client.py", line 1012, in __aenter__
    self._resp = await self._coro
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/client.py", line 357, in _request
    raise Ru

Handle Attachment {'connection_id': 'eda8c739-430f-4c55-8aeb-ee9d0101a2f0', 'message_id': 'bfba1e2c-5f93-4228-bee0-bee1b9af2347', 'attachment_message': 'Here is the data you wanted', 'content': 'VGhpcyBpcyBhIHRlc3QgZmlsZSB0byBhdHRhY2gu', 'filename': 'test_file.txt', 'state': 'received'} eda8c739-430f-4c55-8aeb-ee9d0101a2f0
